In [ ]:
%%capture
dry_run = False
!pip install ../input/kornialoftr/kornia-0.6.4-py2.py3-none-any.whl
!pip install ../input/kornialoftr/kornia_moons-0.1.9-py3-none-any.whl

In [ ]:
%matplotlib inline

import os
import csv
import random
from glob import glob
from tqdm import tqdm
from collections import namedtuple

import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch
import torchvision.transforms as transforms

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import kornia
from kornia_moons.feature import *
import kornia as K
import kornia.feature as KF
import gc
import pydegensac

import sys
import time

sys.path.append("../input/")
sys.path.append("../input/super-glue-pretrained-network")

from models.matching import Matching as Matching_SuperGlue
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

In [ ]:
# Check which GPUs I am assigned to
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

## General Helper Functions

In [ ]:
src = '/kaggle/input/image-matching-challenge-2022/'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]


def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])


def load_torch_image(device, fname=None, local_image=None, size=840.0):
    # If the image is already in memory
    if local_image is None:
        img = cv2.imread(fname)
    else:
        img = np.copy(local_image)
        
    if size == -1:
        scale = 1
    else:
        scale = float(size) / float(max(img.shape[0], img.shape[1]))
    
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))
    img = K.image_to_tensor(img, False).float() /255.0
    img = K.color.bgr_to_rgb(img)
    
    # the scale value here is the new_size / old_size, different from the original SuperGlue 
    return img.to(device), scale

test_samples_df = pd.DataFrame(test_samples, columns=["sample_id", "batch_id", "image_0_id", "image_1_id"])
test_samples_df

## Load SuperGlue

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

resize = [-1, ] # resize = [-1, ] means no resize
# resize = 840
resize_float = True

config = {
    "superpoint": {
        # "nms_radius": 4,
        "nms_radius": 4,
        "keypoint_threshold": 0.005,
        "max_keypoints": 2048
    },
    "superglue": {
        "weights": "outdoor",
        "sinkhorn_iterations": 150,
        "match_threshold": 0.2,
    }
}
matcher_SG = Matching_SuperGlue(config).eval().to(device)

## Load LoFTR

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

matcher_LoFTR = KF.LoFTR(pretrained=None)
matcher_LoFTR.load_state_dict(torch.load("../input/kornialoftr/loftr_outdoor.ckpt")['state_dict'])
# matcher.load_state_dict(torch.load("../input/kornialoftr/epoch12-auc50.931-auc100.966-auc200.983.ckpt")['state_dict'])
# matcher.load_state_dict(torch.load("../input/kornialoftr/epoch3-auc100.960.ckpt")['state_dict'])
matcher_LoFTR = matcher_LoFTR.to(device).eval()

## Functions to extract the keypoints and matches from different models

In [ ]:
def get_keypoints_with_conf_LoFTR(image_1, image_2, matcher, img_resize=840.0, conf_th=[0.75, 0.5, 0.25, 0], num_keypoints=1000, take_all=False):
                          
    # the scale value here is the new_size / old_size, different from SuperGlue, to make it the same, take the inverse
    image_1_tensor, scale_1 = load_torch_image(device, fname=None, local_image=image_1, size=img_resize)
    image_2_tensor, scale_2 = load_torch_image(device, fname=None, local_image=image_2, size=img_resize)
    scale_1 = float(1.0 / float(scale_1))
    scale_2 = float(1.0 / float(scale_2))
    
    input_dict = {"image0": K.color.rgb_to_grayscale(image_1_tensor), 
                  "image1": K.color.rgb_to_grayscale(image_2_tensor)}
    
    with torch.no_grad():
        correspondences = matcher(input_dict)
        
    mkpts0_LoFTR = correspondences['keypoints0'].cpu().numpy()
    mkpts1_LoFTR = correspondences['keypoints1'].cpu().numpy()
    conf = correspondences['confidence'].cpu().numpy()
    # print("initial number of LoFTR points: " + str(len(mkpts0_LoFTR)))
    
    # This is usually for SE2-LoFTR
    if take_all == True:
        return mkpts0_LoFTR, mkpts1_LoFTR, np.mean(conf), 0, scale_1, scale_2
    
    mkpts0_LoFTR_conf_0 = mkpts0_LoFTR[conf > conf_th[0]]
    mkpts1_LoFTR_conf_0 = mkpts1_LoFTR[conf > conf_th[0]]
    mkpts0_LoFTR_conf_1 = mkpts0_LoFTR[conf > conf_th[1]]
    mkpts1_LoFTR_conf_1= mkpts1_LoFTR[conf > conf_th[1]]
    mkpts0_LoFTR_conf_2 = mkpts0_LoFTR[conf > conf_th[2]]
    mkpts1_LoFTR_conf_2 = mkpts1_LoFTR[conf > conf_th[2]]
    mkpts0_LoFTR_all = mkpts0_LoFTR[conf >= conf_th[3]]
    mkpts1_LoFTR_all = mkpts1_LoFTR[conf >= conf_th[3]]
    
    # Use a progressive method to select the confidence threshold
    #  If there are too many keypoints, take high confidence, otherwise take low
    num_bin_1 = len(mkpts0_LoFTR_conf_0)
    num_bin_2 = len(mkpts0_LoFTR_conf_1) - num_bin_1
    num_bin_3 = len(mkpts0_LoFTR_conf_2) - num_bin_2 - num_bin_1
    num_bin_4 = len(mkpts0_LoFTR_all) - num_bin_3 - num_bin_2 - num_bin_1
    
    largest_bin_index = np.argmax(np.array([num_bin_1, num_bin_2, num_bin_3, num_bin_4]))
    conf_th_final = conf_th[largest_bin_index]
    
    mkpts0_LoFTR_final = mkpts0_LoFTR[conf > conf_th_final]
    mkpts1_LoFTR_final = mkpts1_LoFTR[conf > conf_th_final]
    conf_mean = np.mean(conf[conf > conf_th_final])
    
    if len(mkpts0_LoFTR_final) <= 7:
        mkpts0_LoFTR_final = mkpts0_LoFTR_all
        mkpts1_LoFTR_final = mkpts1_LoFTR_all
        conf_mean = np.mean(conf)
    
    # If we don't use SE2-LoFTR, limit the number of matched keypoints
    # Since experiments show that sometimes LoFTR can create an excessive amount of matching points
    if len(mkpts0_LoFTR_final) > num_keypoints:
        conf_final = conf[conf > conf_th_final]
        conf_argsorted = np.argsort(conf_final)
        selected_indices = conf_argsorted[-num_keypoints:]
        mkpts0_LoFTR_final = mkpts0_LoFTR_final[selected_indices]
        mkpts1_LoFTR_final = mkpts1_LoFTR_final[selected_indices]
        
    # print("final number of LoFTR points: " + str(len(mkpts1_LoFTR_final)))
    return mkpts0_LoFTR_final, mkpts1_LoFTR_final, conf_mean, conf_th_final, scale_1, scale_2


def get_keypoints_with_conf_SG(image_fpath_0, image_fpath_1, matcher, resize, resize_float, conf_th=[0.75, 0.5, 0.25, 0], take_all=False):
    
    # scale = original_size / new_size, different from the original SuperGlue. 
    image_0, inp_0, scales_0 = read_image(image_fpath_0, device, resize, 0, resize_float)
    image_1, inp_1, scales_1 = read_image(image_fpath_1, device, resize, 0, resize_float)

    input_dict = {"image0": inp_0, "image1": inp_1}

    with torch.no_grad():
        pred_SG = matcher(input_dict)
        
    pred_SG = {k: v[0].detach().cpu().numpy() for k, v in pred_SG.items()}
    kpts0_SG, kpts1_SG = pred_SG["keypoints0"], pred_SG["keypoints1"]
    # matches mask are different "matches0" and "matches1" since the number of keypoints are different
    # but the valid keypoints after applying the mask will be the same (actually still different, probably a bug)
    matches_mask_0_SG, conf_0 = pred_SG["matches0"], pred_SG["matching_scores0"]
    
    valid_0 = matches_mask_0_SG > -1
    mkpts0_SG = kpts0_SG[valid_0]
    mkpts1_SG = kpts1_SG[matches_mask_0_SG[valid_0]]
    conf_0 = conf_0[valid_0]
    conf = conf_0
    
    # print("initial number of SG points: " + str(len(mkpts0_SG)))
    # This is usually for SE2-LoFTR
    if take_all == True:
        return mkpts0_SG, mkpts1_SG, np.mean(conf), 0, scale_1, scale_2
    
    mkpts0_SG_conf_0 = mkpts0_SG[conf > conf_th[0]]
    mkpts1_SG_conf_0 = mkpts1_SG[conf > conf_th[0]]
    mkpts0_SG_conf_1 = mkpts0_SG[conf > conf_th[1]]
    mkpts1_SG_conf_1= mkpts1_SG[conf > conf_th[1]]
    mkpts0_SG_conf_2 = mkpts0_SG[conf > conf_th[2]]
    mkpts1_SG_conf_2 = mkpts1_SG[conf > conf_th[2]]
    mkpts0_SG_all = mkpts0_SG[conf >= conf_th[3]]
    mkpts1_SG_all = mkpts1_SG[conf >= conf_th[3]]
    
    # Use a progressive method to select the confidence threshold
    #  If there are too many keypoints, take high confidence, otherwise take low
    num_bin_1 = len(mkpts0_SG_conf_0)
    num_bin_2 = len(mkpts0_SG_conf_1) - num_bin_1
    num_bin_3 = len(mkpts0_SG_conf_2) - num_bin_2 - num_bin_1
    num_bin_4 = len(mkpts0_SG_all) - num_bin_3 - num_bin_2 - num_bin_1
    
    largest_bin_index = np.argmax(np.array([num_bin_1, num_bin_2, num_bin_3, num_bin_4]))
    conf_th_final = conf_th[largest_bin_index]
    
    mkpts0_SG_final = mkpts0_SG[conf > conf_th_final]
    mkpts1_SG_final = mkpts1_SG[conf > conf_th_final]
    conf_mean = np.mean(conf[conf > conf_th_final])
    
    if len(mkpts0_SG_final) <= 7:
        mkpts0_SG_final = mkpts0_SG_all
        mkpts1_SG_final = mkpts1_SG_all
        conf_mean = np.mean(conf)

    # print("final number of SG points: " + str(len(mkpts0_SG_final)))
    return mkpts0_SG_final, mkpts1_SG_final, conf_mean, conf_th_final, scales_0, scales_1

## Create the dictionary for different scaling factors for different scenes

In [ ]:
## Create a dictionary for scaling factors and pair's calibration data of different scenes
src = '../input/image-matching-challenge-2022/train'

def create_scaling_dict(src):
    scaling_dict = {}
    with open(f'{src}/scaling_factors.csv') as f:
        reader = csv.reader(f, delimiter=',')
        for i, row in enumerate(reader):
            # Skip header.
            if i == 0:
                continue
            scaling_dict[row[0]] = float(row[1])

    print(f'Scaling factors: {scaling_dict}')
    return scaling_dict

def LoadCalibration(filename):
    '''Load calibration data (ground truth) from the csv file.'''
    
    calib_dict = {}
    with open(filename, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for i, row in enumerate(reader):
            # Skip header.
            if i == 0:
                continue

            camera_id = row[0]
            K = np.array([float(v) for v in row[1].split(' ')]).reshape([3, 3])
            R = np.array([float(v) for v in row[2].split(' ')]).reshape([3, 3])
            T = np.array([float(v) for v in row[3].split(' ')])
            calib_dict[camera_id] = Gt(K=K, R=R, T=T)
    
    return calib_dict

scaling_dict = create_scaling_dict(src)

## Helper functions to build the evaluation pipeline

In [ ]:
# Some useful functions and definitions. You can skip this for now.

# A named tuple containing the intrinsics (calibration matrix K) and extrinsics (rotation matrix R, translation vector T) for a given camera.
Gt = namedtuple('Gt', ['K', 'R', 'T'])

# A small epsilon.
eps = 1e-15

def ReadCovisibilityData(filename):
    covisibility_dict = {}
    with open(filename) as f:
        reader = csv.reader(f, delimiter=',')
        for i, row in enumerate(reader):
            # Skip header.
            if i == 0:
                continue
            covisibility_dict[row[0]] = float(row[1])

    return covisibility_dict


# Project the keypoints' coordinates to the image plane 
# (i.e., to the normalized coordinate system as shown on a digital display)
def NormalizeKeypoints(keypoints, K):
    C_x = K[0, 2]
    C_y = K[1, 2]
    f_x = K[0, 0]
    f_y = K[1, 1]
    keypoints = (keypoints - np.array([[C_x, C_y]])) / np.array([[f_x, f_y]])
    return keypoints


def ComputeEssentialMatrix(F, K1, K2, kp1, kp2):
    '''Compute the Essential matrix from the Fundamental matrix, given the calibration matrices. 
       Note that we ask participants to estimate F, i.e., without relying on known intrinsics.'''
    
    # Warning! Old versions of OpenCV's RANSAC could return multiple F matrices, encoded as a single matrix size 6x3 or 9x3, rather than 3x3.
    # We do not account for this here, as the modern RANSACs do not do this:
    # https://opencv.org/evaluating-opencvs-new-ransacs
    assert F.shape[0] == 3, 'Malformed F?'

    # Use OpenCV's recoverPose to solve the cheirality check:
    # https://docs.opencv.org/4.5.4/d9/d0c/group__calib3d.html#gadb7d2dfcc184c1d2f496d8639f4371c0
    E = np.matmul(np.matmul(K2.T, F), K1).astype(np.float64)
    
    kp1n = NormalizeKeypoints(kp1, K1)
    kp2n = NormalizeKeypoints(kp2, K2)
    num_inliers, R, T, mask = cv2.recoverPose(E, kp1n, kp2n)

    return E, R, T


def ArrayFromCvKps(kps):
    '''Convenience function to convert OpenCV keypoints into a simple numpy array.'''
    return np.array([kp.pt for kp in kps])


def QuaternionFromMatrix(matrix):
    '''Transform a rotation matrix into a quaternion.'''

    M = np.array(matrix, dtype=np.float64, copy=False)[:4, :4]
    m00 = M[0, 0]
    m01 = M[0, 1]
    m02 = M[0, 2]
    m10 = M[1, 0]
    m11 = M[1, 1]
    m12 = M[1, 2]
    m20 = M[2, 0]
    m21 = M[2, 1]
    m22 = M[2, 2]

    K = np.array([[m00 - m11 - m22, 0.0, 0.0, 0.0],
              [m01 + m10, m11 - m00 - m22, 0.0, 0.0],
              [m02 + m20, m12 + m21, m22 - m00 - m11, 0.0],
              [m21 - m12, m02 - m20, m10 - m01, m00 + m11 + m22]])
    K /= 3.0

    # The quaternion is the eigenvector of K that corresponds to the largest eigenvalue.
    w, V = np.linalg.eigh(K)
    q = V[[3, 0, 1, 2], np.argmax(w)]

    if q[0] < 0:
        np.negative(q, q)

    return q

def DrawMatches(im1, im2, kp1, kp2, matches, axis=1, margin=0, background=0, linewidth=2):
    '''Draw keypoints and matches.'''
    
    composite, v_offset, h_offset = BuildCompositeImage(im1, im2, axis, margin, background)

    # Draw all keypoints.
    for coord_a, coord_b in zip(kp1, kp2):
        composite = cv2.drawMarker(composite, (int(coord_a[0] + h_offset[0]), int(coord_a[1] + v_offset[0])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
        composite = cv2.drawMarker(composite, (int(coord_b[0] + h_offset[1]), int(coord_b[1] + v_offset[1])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
    
    # Draw matches, and highlight keypoints used in matches.
    for idx_a, idx_b in matches:
        composite = cv2.drawMarker(composite, (int(kp1[idx_a, 0] + h_offset[0]), int(kp1[idx_a, 1] + v_offset[0])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.drawMarker(composite, (int(kp2[idx_b, 0] + h_offset[1]), int(kp2[idx_b, 1] + v_offset[1])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.line(composite,
                             tuple([int(kp1[idx_a][0] + h_offset[0]),
                                   int(kp1[idx_a][1] + v_offset[0])]),
                             tuple([int(kp2[idx_b][0] + h_offset[1]),
                                   int(kp2[idx_b][1] + v_offset[1])]), color=(0, 0, 255), thickness=1)
    return composite

    
def ComputeErrorForOneExample(q_gt, T_gt, q, T, scale):
    '''Compute the error metric for a single example.
    The function returns two errors, over rotation and translation. 
    These are combined at different thresholds by ComputeMaa in order to compute the mean Average Accuracy.'''
    
    q_gt_norm = q_gt / (np.linalg.norm(q_gt) + eps)
    q_norm = q / (np.linalg.norm(q) + eps)

    loss_q = np.maximum(eps, (1.0 - np.sum(q_norm * q_gt_norm)**2))
    err_q = np.arccos(1 - 2 * loss_q)

    # Apply the scaling factor for this scene.
    T_gt_scaled = T_gt * scale
    T_scaled = T * np.linalg.norm(T_gt) * scale / (np.linalg.norm(T) + eps)

    err_t = min(np.linalg.norm(T_gt_scaled - T_scaled), np.linalg.norm(T_gt_scaled + T_scaled))

    return err_q * 180 / np.pi, err_t


def ComputeMaa(err_q, err_t, thresholds_q, thresholds_t):
    '''Compute the mean Average Accuracy at different tresholds, for one scene.'''
    
    assert len(err_q) == len(err_t)
    
    acc, acc_q, acc_t = [], [], []
    for th_q, th_t in zip(thresholds_q, thresholds_t):
        acc += [(np.bitwise_and(np.array(err_q) < th_q, np.array(err_t) < th_t)).sum() / len(err_q)]
        acc_q += [(np.array(err_q) < th_q).sum() / len(err_q)]
        acc_t += [(np.array(err_t) < th_t).sum() / len(err_t)]

    return np.mean(acc), np.array(acc), np.array(acc_q), np.array(acc_t)

## Iterate over all scenes 

In [ ]:
# Compute the metric for each scene, and then average it over all scenes.
# Cap the number of image pairs for each scene to 50, and show one qualitative example per scene.
show_images = True
num_show_images = 10
max_pairs_per_scene = 10
verbose = True
src = '../input/image-matching-challenge-2022/train'

# Use two different sets of thresholds over rotation and translation. Do not change this 
# -- these are the values used by the scoring back-end.
thresholds_q = np.linspace(1, 10, 10)   # threshold for rotation values
thresholds_t = np.geomspace(0.2, 5, 10) # threshold for translation values

# Save the per-sample errors and the accumulated metric to dictionaries, for later inspection.
errors = {scene: {} for scene in scaling_dict.keys()}
mAA = {scene: {} for scene in scaling_dict.keys()}

# matcher_LoFTR = matcher_SE2_LoFTR

# Instantiate the matcher.
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# scene = 'colosseum_exterior'
for scene in scaling_dict.keys():
    covisibility_dict = ReadCovisibilityData(f'{src}/{scene}/pair_covisibility.csv')
    
    pairs = [pair for pair, covis in covisibility_dict.items() if covis >= 0.1]
    print(f'-- Processing scene "{scene}": found {len(pairs)} pairs \
            (will keep {min(len(pairs), max_pairs_per_scene)})', flush=True)
    
    # Subsample the pairs. Note that they are roughly sorted by difficulty (easy ones first), 
    # so we shuffle them beforehand: results would be misleading otherwise.
    # random.shuffle(pairs)
    #
    # You can change the parameters here to include more image pairs for validation, 
    # for illustration purpose, I am only putting one pair for each scene here
    pairs = pairs[0:1]
    print("len(pairs): " + str(len(pairs)))
    
    # Extract the images in these pairs (we don't need to load images we will not use).
    pair_ids = []
    for pair in pairs:
        cur_ids = pair.split('-')
        assert cur_ids[0] > cur_ids[1]
        pair_ids += cur_ids
    pair_ids = list(set(pair_ids))
    
    # Load ground truth data.
    calib_dict = LoadCalibration(f'{src}/{scene}/calibration.csv')
    
    # Load images one by one (not in pairs)
    images_path_dict = {}
    for id in tqdm(pair_ids):
        images_path_dict[id] = f'{src}/{scene}/images/{id}.jpg'
    

    # print(images_path_dict)
    # Process the pairs.
    for counter, pair in enumerate(pairs):
        id1, id2 = pair.split('-')
        # print(id1)
        # Compute matches to get the keypoints' coordinates
        image_fpath_1 = images_path_dict[id1]
        image_fpath_2 = images_path_dict[id2]
        image_1 = cv2.imread(image_fpath_1)
        image_2 = cv2.imread(image_fpath_2)
        image_1_tensor, scale = load_torch_image(device, fname=None, local_image=image_1, size=-1)
        image_2_tensor, scale = load_torch_image(device, fname=None, local_image=image_2, size=-1)
        
        img1_max_dim = max(image_1.shape[0], image_1.shape[1])
        img2_max_dim = max(image_2.shape[0], image_2.shape[1])
        max_dim = max(img1_max_dim, img2_max_dim)
        if max_dim > 1250:
            max_dim = 1250
        if max_dim < 750:
            max_dim = 750
            
        input_dict = {"image0": K.color.rgb_to_grayscale(image_1_tensor), 
                      "image1": K.color.rgb_to_grayscale(image_2_tensor)}

        # First use LoFTR to get a coarse match (I only use one image size for illustration purpose)
        conf_th = [0.75, 0.5, 0.25, 0]
#         mkpts0_LoFTR, mkpts1_LoFTR, conf_mean_LoFTR, conf_th_LoFTR, scale_1_LoFTR, scale_2_LoFTR = \
#                                     get_keypoints_with_conf_LoFTR(image_1, image_2, matcher_LoFTR,\
#                                     img_resize=-1, conf_th=conf_th, take_all=False)  
        
#         mkpts0_LoFTR_resize1, mkpts1_LoFTR_resize1, conf_mean_LoFTR_resize1, conf_th_LoFTR_resize1,\
#                 scale_1_LoFTR_resize1, scale_2_LoFTR_resize1 = get_keypoints_with_conf_LoFTR(image_1, \
#                 image_2, matcher_LoFTR, img_resize=max_dim*1.2, conf_th=conf_th, take_all=False)

        mkpts0_LoFTR_resize2, mkpts1_LoFTR_resize2, conf_mean_LoFTR_resize2, conf_th_LoFTR_resize2, \
                scale_1_LoFTR_resize2, scale_2_LoFTR_resize2 = get_keypoints_with_conf_LoFTR(image_1, \
                image_2, matcher_LoFTR, img_resize=840, conf_th=conf_th, take_all=False)
        
        
        # Second use SuperGlue to get a coarse match (I only use one image size for illustration purpose))
        mkpts0_SG, mkpts1_SG, conf_mean_SG, conf_th_SG, scale_1_SG, scale_2_SG = \
                        get_keypoints_with_conf_SG(image_fpath_1,image_fpath_2, matcher_SG, \
                        resize=[-1, ], resize_float=resize_float, conf_th=conf_th, take_all=False)

#         mkpts0_SG_resize1, mkpts1_SG_resize1, conf_mean_SG_resize1, conf_th_SG_resize1, scale_1_SG_resize1, \
#                         scale_2_SG_resize1 = get_keypoints_with_conf_SG(image_fpath_1,image_fpath_2, matcher_SG, \
#                         resize=[max_dim*1.5, ], resize_float=resize_float, conf_th=conf_th, take_all=False)

#         mkpts0_SG_resize2, mkpts1_SG_resize2, conf_mean_SG_resize2, conf_th_SG_resize2, scale_1_SG_resize2, \
#                         scale_2_SG_resize2 = get_keypoints_with_conf_SG(image_fpath_1,image_fpath_2, matcher_SG, \
#                         resize=[max_dim*0.6, ], resize_float=resize_float, conf_th=conf_th, take_all=False)
        
        # conf_th_mean_SG = np.mean([conf_th_SG, conf_th_SG_resize1, conf_th_SG_resize2])
        
        # mkpts0_LoFTR_ns = mkpts0_LoFTR * scale_1_LoFTR
        # mkpts0_LoFTR_s1 = mkpts0_LoFTR_resize1 * scale_1_LoFTR_resize1
        mkpts0_LoFTR_s2 = mkpts0_LoFTR_resize2 * scale_1_LoFTR_resize2
        # mkpts1_LoFTR_ns = mkpts1_LoFTR * scale_2_LoFTR
        # mkpts1_LoFTR_s1 = mkpts1_LoFTR_resize1 * scale_2_LoFTR_resize1
        mkpts1_LoFTR_s2 = mkpts1_LoFTR_resize2 * scale_2_LoFTR_resize2
        
        mkpts0_SG_ns = mkpts0_SG * scale_1_SG
        # mkpts0_SG_s1 = mkpts0_SG_resize1 * scale_1_SG_resize1
        # mkpts0_SG_s2 = mkpts0_SG_resize2 * scale_1_SG_resize2
        mkpts1_SG_ns = mkpts1_SG * scale_2_SG
        # mkpts1_SG_s1 = mkpts1_SG_resize1 * scale_2_SG_resize1
        # mkpts1_SG_s2 = mkpts1_SG_resize2 * scale_2_SG_resize2
        
        
#         mkpts0_combined = np.concatenate((mkpts0_LoFTR_s1, mkpts0_LoFTR_s2, \
#                                           mkpts0_SG_ns, mkpts0_SG_s1, mkpts0_SG_s2), axis=0)
#         mkpts1_combined = np.concatenate((mkpts1_LoFTR_s1, mkpts1_LoFTR_s2, \
#                                           mkpts1_SG_ns, mkpts1_SG_s1, mkpts1_SG_s2), axis=0)
        
        mkpts0_combined = np.concatenate((mkpts0_LoFTR_s2, mkpts0_SG_ns), axis=0)                 
        mkpts1_combined = np.concatenate((mkpts1_LoFTR_s2, mkpts1_SG_ns), axis=0)                                      

        # Get the F-matrix 
        if len(mkpts0_combined) > 7:
            F, inliers = cv2.findFundamentalMat(mkpts0_combined, mkpts1_combined, cv2.USAC_MAGSAC, 0.2, 0.99999, 250000)
#             F, inliers = pydegensac.findFundamentalMatrix(mkpts0_combined, mkpts1_combined, \
#                                          px_th=0.2, conf=0.99999, max_iters=800000, laf_consistensy_coef=0, \
#                                          error_type='symm_epipolar', symmetric_error_check=True, enable_degeneracy_check=True)  
            inliers = inliers.squeeze() > 0  
            assert F.shape == (3, 3), 'Malformed F?'
            # F_dict[sample_id] = F  

            print("number of inliers: " + str(np.count_nonzero(inliers)))

        else:
            print("zero F matrix")
            # F_dict[sample_id] = np.zeros((3, 3))
            # continue


        # keypoints' coordinates after RANSAC
        mkpts0_final = mkpts1_combined[inliers]
        mkpts1_final = mkpts1_combined[inliers]

        # Compute the essential matrix.
        E_LoFTR, R_LoFTR, T_LoFTR = ComputeEssentialMatrix(F, calib_dict[id1].K, calib_dict[id2].K, mkpts0_final, mkpts1_final)
        q_LoFTR = QuaternionFromMatrix(R_LoFTR)
        T_LoFTR = T_LoFTR.flatten()

        # Get the relative rotation and translation between these two cameras, given their R and T in the global reference frame.
        # print(str(id1) + str(calib_dict[id1].T))
        # print(str(id2) + str(calib_dict[id2].T))
        R1_gt, T1_gt = calib_dict[id1].R, calib_dict[id1].T.reshape((3, 1))
        R2_gt, T2_gt = calib_dict[id2].R, calib_dict[id2].T.reshape((3, 1))
        dR_gt = np.dot(R2_gt, R1_gt.T)
        dT_gt = (T2_gt - np.dot(dR_gt, T1_gt)).flatten()
        q_gt = QuaternionFromMatrix(dR_gt)
        q_gt = q_gt / (np.linalg.norm(q_gt) + eps)

        # Compute the error for this example.
        err_q_LoFTR, err_t_LoFTR = ComputeErrorForOneExample(q_gt, dT_gt, q_LoFTR, T_LoFTR, scaling_dict[scene])
        errors[scene][pair] = [err_q_LoFTR, err_t_LoFTR]
        
        # torch.cuda.empty_cache()
        gc.collect()
        

        # Plot the resulting matches and the pose error.
        if verbose or (show_images and counter < num_show_images):
            print(f'{pair}, err_q={(err_q_LoFTR):.02f} (deg), err_t={(err_t_LoFTR):.02f} (m)', flush=True)
        if show_images and counter < num_show_images:
            draw_LAF_matches(
                KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0_combined).view(1,-1, 2),
                                            torch.ones(mkpts0_combined.shape[0]).view(1,-1, 1, 1),
                                            torch.ones(mkpts0_combined.shape[0]).view(1,-1, 1)),

                KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1_combined).view(1,-1, 2),
                                            torch.ones(mkpts1_combined.shape[0]).view(1,-1, 1, 1),
                                            torch.ones(mkpts1_combined.shape[0]).view(1,-1, 1)),
                torch.arange(mkpts1_combined.shape[0]).view(-1,1).repeat(1,2),
                K.tensor_to_image(image_1_tensor.detach().cpu()),
                K.tensor_to_image(image_2_tensor.detach().cpu()),
                inliers,
                draw_dict={'inlier_color': (0.2, 1, 0.2),
                          'tentative_color': None, 
                          'feature_color': (0.2, 0.5, 1), 'vertical': False})

    # Histogram the errors over this scene.
    mAA[scene] = ComputeMaa([v[0] for v in errors[scene].values()], [v[1] \
                              for v in errors[scene].values()], thresholds_q, thresholds_t)
    print()
    print(f'Mean average Accuracy on "{scene}": {mAA[scene][0]:.05f}')
    print()

print()
print('------- SUMMARY -------')
print()
for scene in scaling_dict.keys():
    print(f'-- Mean average Accuracy on "{scene}": {mAA[scene][0]:.05f}')
print()
print(f'Mean average Accuracy on dataset: {np.mean([mAA[scene][0] for scene in mAA]):.05f}')